# Results Neurocomputing/ESANN 2024

This notebook contains the code to analyse the results of the 
Neurocomputing/ESANN 2024 paper, and it is responsible for generating
the figures and tables in the paper.

The notebook is organised as follows:

1. The first section contains imports, constants, helper functions and load the 
    data.

2. We show that the dict-wisard has competitive performance with the 
    classical machine learning algorithms.

## 1. General constants, hhelper functions, and data loading

Imports, global constants and packages' configuration.

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from typing import List, Union
import plotly.graph_objects as go
import plotly.express as px
import json

from utils import write_figure, write_latex_table, aggregate_mean_std

In [2]:
# Configs
pd.set_option("display.float_format", lambda x: "%.4f" % x)

# ---------- Paths -------------
# -- Inputs
datasets_info_path = Path("datasets_info.json")

results_sklearn_path = [
    Path("results_knn_folded.csv"),
    Path("results_mlp_folded.csv"),
    Path("results_mlp2_folded.csv"),
    Path("results_svm_folded.csv"),
    Path("results_rf_folded.csv"),
]

results_wisard_path = Path("results_wisard_folded.csv")

### Read inputs and create a full dataframe

1. Read the datasets specifications (`dataset_info`)
2. Read the wisard results (`wisard_results`)
3. Read the sklearn results (`sklearn_results`)
4. Create a results dataframe, mergind dataset_info, wisard_results and sklearn_results

#### Dataset information

In [3]:
# Datasets information
datasets_info = pd.read_json(datasets_info_path, orient="index").reset_index(drop=True)
datasets_info.rename(columns={"name": "dataset_name"}, inplace=True)
datasets_info.head(n=3)

,dataset_name,size,features,num_classes,train_size,test_size,balanced,metric
0,breast_cancer,141416,30,3,398,171,False,f1 weighted
1,dry_bean,1773910,16,7,10888,2723,False,f1 weighted
2,glass,17413,9,24,149,65,False,f1 weighted


In [4]:
info = datasets_info[
    ["dataset_name", "features", "size", "num_classes", "balanced"]
]
info.loc[:, "size"] = info["size"] / 1024

info = info.rename(
    columns={
        "dataset_name": "Dataset",
        "features": "Features",
        "size": "Size (KB)",
        "num_classes": "Classes",
        "balanced": "Is Balanced?",
    }
)

latex_str = info.to_latex(
    index=False,
    escape=True,
    caption="Datasets information",
    label="tab:datasets_info",
    float_format="%.2f",
)

write_latex_table("datasets_info.tex", latex_str)

Table written to: tables/datasets_info.tex


/tmp/ipykernel_2961821/3004690570.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info.loc[:, "size"] = info["size"] / 1024
/tmp/ipykernel_2961821/3004690570.py:16: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_str = info.to_latex(


#### Wisard results

Read and parse wisard result to `wisard_results` dataframe.

**Note**: The `wisard_results` already has aggregated results for each dataset.

In [5]:
def parse_wisard_config_name(row) -> str:
    """Given a row, parse the name of configuration.

    Parameters
    ----------
    row : pd.Series
        The row of the dataframe.

    Returns
    -------
    str
        The name of the configuration.
    """

    names = []
    if not pd.isna(row["num_hitters"]):
        names.append(f"NR: {row['num_hitters']}")
    if not pd.isna(row["width"]):
        names.append(f"W: {row['width']}")
    if not pd.isna(row["depth"]):
        names.append(f"D: {row['depth']}")
    if not pd.isna(row["capacity"]):
        names.append(f"C: {row['capacity']}")
    if not pd.isna(row["bucket_size"]):
        names.append(f"BS: {row['bucket_size']}")
    if not pd.isna(row["threshold"]):
        names.append(f"T: {row['threshold']}")
    if not pd.isna(row["est_elements"]):
        names.append(f"EST: {row['est_elements']}")
    if not pd.isna(row["false_positive_rate"]):
        names.append(f"FPR: {row['false_positive_rate']}")

    if names:
        names = ", ".join(names)
        return f"{row['ram']} ({names})"
    else:
        return row["ram"]


# --- Read results and add a column with the name of the configuration ---
wisard_results = pd.read_csv(results_wisard_path).drop_duplicates()

# --- Add useful columns ---
wisard_results["tuple_size"] = (
    wisard_results["resolution"] / wisard_results["tuple_resolution_factor"]
)
wisard_results["config_name"] = wisard_results.apply(
    parse_wisard_config_name, axis=1
)

# --- Select the columns of interest ---
wisard_results = wisard_results[
    [
        "dataset_name",
        "config_name",
        "val_accuracy_mean",
        "val_accuracy_std",
        "val_f1 weighted_mean",
        "val_f1 weighted_std",
        "val_model size_mean",
        "val_model size_std",
        "val_ties_mean",
        "val_ties_std",
        "test_accuracy_mean",
        "test_accuracy_std",
        "test_f1 weighted_mean",
        "test_f1 weighted_std",
        "test_model size_mean",
        "test_model size_std",
        "test_ties_mean",
        "test_ties_std",
        "tuple_size",
        "encoder",
        "resolution",
        "bleach",
        "rams per discriminator",
        "ram",
    ]
]

# --- Rename columns ---
wisard_results = wisard_results.rename(
    columns={
        "dataset_name": "dataset",
        "ram": "model",
        "test_ties_mean": "ties",
        "test_ties_std": "ties_std",
        "test_accuracy_mean": "accuracy",
        "test_accuracy_std": "accuracy_std",
        "test_f1 weighted_mean": "f1",
        "test_f1 weighted_std": "f1_std",
        "test_model size_mean": "model_size",
        "test_model size_std": "model_size_std",
    }
)

# --- Add model column and drop duplicates ---
wisard_results["model"] = "Wisard"
wisard_results.drop_duplicates(inplace=True)

# Split dataset name from fold
wisard_results[["dataset", "fold"]] = wisard_results["dataset"].str.split(
    "_fold_", expand=True
)
wisard_results["fold"] = wisard_results["fold"].astype(int)
wisard_results.sample(n=2)

,dataset,config_name,val_accuracy_mean,val_accuracy_std,val_f1 weighted_mean,val_f1 weighted_std,val_model size_mean,val_model size_std,val_ties_mean,val_ties_std,...,model_size_std,ties,ties_std,tuple_size,encoder,resolution,bleach,rams per discriminator,model,fold
3968,sepsis,Dict,0.8410,0.0652,0.8520,0.0340,16216.3333,225.1301,1966.0000,1502.9280,...,307.7087,4851.6667,1389.8514,23.0000,thermometer,46,1354,6,Wisard,0
2768,glass,"CountMinSketch (W: 13.0, D: 4.0)",0.3714,0.0000,0.2012,0.0000,12096.0000,0.0000,0.0000,0.0000,...,0.0000,19.3333,16.7398,33.0000,thermometer,33,48,9,Wisard,2


In [6]:
dict_wisard_results = wisard_results[(wisard_results["config_name"] == "Dict")]
# dict_wisard_results = wisard_results

lines = []
for (dataset, fold), dataset_df in dict_wisard_results.groupby(["dataset", "fold"]):
    metric_name = datasets_info.loc[datasets_info["dataset_name"] == dataset, "metric"].iloc[0]
    line = dataset_df.sort_values(by=f"val_{metric_name}_mean", ascending=False).iloc[0]
    lines.append(line)

dict_wisard_results = pd.DataFrame(lines)

dict_wisard_results = aggregate_mean_std(
    dict_wisard_results, 
    group_by=["dataset"],
    keys_to_aggregate=["accuracy", "f1", "model_size"]
)

dict_wisard_results["model"] = "Dict-Wisard"

# Rearange columns
dict_wisard_results = dict_wisard_results[[
    "dataset",
    "model",
    "accuracy",
    "accuracy_std",
    "f1",
    "f1_std",
    "model_size",
    "model_size_std",
]]

dict_wisard_results

,dataset,model,accuracy,accuracy_std,f1,f1_std,model_size,model_size_std
0,breast_cancer,Dict-Wisard,0.9279,0.0159,0.9274,0.0163,40581.0000,23921.9804
1,dry_bean,Dict-Wisard,0.9011,0.0047,0.9012,0.0043,688523.4000,481035.0196
2,glass,Dict-Wisard,0.6231,0.0194,0.5661,0.0213,37794.8000,21282.6736
3,image_segmentation,Dict-Wisard,0.6540,0.2596,0.6492,0.2615,107542.8000,63761.3701
4,iris,Dict-Wisard,0.9600,0.0513,0.9595,0.0519,3418.0667,3621.7708
5,letter,Dict-Wisard,0.8739,0.0291,0.8751,0.0286,7193661.8667,4057207.8151
6,motion_sense,Dict-Wisard,0.7314,0.0423,0.7181,0.0441,46898528.6667,18848361.5343
7,optical_handwritten,Dict-Wisard,0.9714,0.0083,0.9714,0.0083,8721863.2667,7681296.4388
8,rice,Dict-Wisard,0.9052,0.0115,0.9051,0.0116,24524.9333,25997.3475
9,satimage,Dict-Wisard,0.8890,0.0064,0.8847,0.0070,6512633.0000,4208685.7637


#### Scikit Learn results

Read and parse sklearn result to `sklearn_results` dataframe.

In [7]:
# Read sklearn results and aggregate multiple runs

dfs = []
for results_path in results_sklearn_path:
    df = pd.read_csv(results_path).drop_duplicates()
    model = results_path.stem.split("_")[1]
    if model == "mlp2":
        model = "mlp"
    df["model"] = model
    dfs.append(df)
sklearn_results = pd.concat(dfs)
sklearn_results

sklearn_results = sklearn_results[[
    "dataset_name",
    "model",
    "val_accuracy_mean",
    "val_accuracy_std",
    "val_f1 weighted_mean",
    "val_f1 weighted_std",
    "val_model size_mean",
    "val_model size_std",
    "test_accuracy_mean",
    "test_accuracy_std",
    "test_f1 weighted_mean",
    "test_f1 weighted_std",
    "test_model size_mean",
    "test_model size_std",
]]

sklearn_results = sklearn_results.rename(columns={
    "dataset_name": "dataset",
    "test_accuracy_mean": "accuracy",
    "test_accuracy_std": "accuracy_std",
    "test_f1 weighted_mean": "f1",
    "test_f1 weighted_std": "f1_std",
    "test_model size_mean": "model_size",
    "test_model size_std": "model_size_std", 
})

# Split dataset name from fold
sklearn_results[["dataset", "fold"]] = sklearn_results["dataset"].str.split(
    "_fold_", expand=True
)
sklearn_results["fold"] = sklearn_results["fold"].astype(int)
sklearn_results.sample(n=2)

,dataset,model,val_accuracy_mean,val_accuracy_std,val_f1 weighted_mean,val_f1 weighted_std,val_model size_mean,val_model size_std,accuracy,accuracy_std,f1,f1_std,model_size,model_size_std,fold
306,breast_cancer,knn,0.9011,0.0000,0.8993,0.0000,90944.0000,0.0000,0.9211,0.0000,0.9194,0.0000,90944.0000,0.0000,3
618,optical_handwritten,svm,0.7667,0.0000,0.8088,0.0000,2111983.0000,0.0000,0.7286,0.0000,0.7797,0.0000,2111983.0000,0.0000,2


In [8]:
lines = []
for (dataset,fold,moel), dataset_df in sklearn_results.groupby(["dataset", "fold", "model"]):
    metric_name = datasets_info.loc[datasets_info["dataset_name"] == dataset, "metric"].iloc[0]
    # if metric_name == "f1 weighted":
    #     metric_name = "f1"
    # # Model with hishest
    # line = dataset_df.sort_values(by=metric_name, ascending=False).iloc[0]
    line = dataset_df.sort_values(by=f"val_{metric_name}_mean", ascending=False).iloc[0]
    lines.append(line)
    
sklearn_results = pd.DataFrame(lines)

sklearn_results = aggregate_mean_std(
    sklearn_results, 
    group_by=["dataset", "model"],
    keys_to_aggregate=["accuracy", "f1", "model_size"]
)

sklearn_results = sklearn_results[[
    "dataset",
    "model",
    "accuracy",
    "accuracy_std",
    "f1",
    "f1_std",
    "model_size",
    "model_size_std",
]]

sklearn_results

,dataset,model,accuracy,accuracy_std,f1,f1_std,model_size,model_size_std
0,breast_cancer,knn,0.9297,0.0255,0.9288,0.0269,129632.8000,52987.9426
1,breast_cancer,mlp,0.9315,0.0224,0.9308,0.0241,123549.4667,180986.9919
2,breast_cancer,rf,0.9554,0.0166,0.9554,0.0167,47683.1333,38508.2852
3,breast_cancer,svm,0.9561,0.0397,0.9552,0.0417,11406.6000,4215.2497
4,dry_bean,knn,0.6769,0.1227,0.6777,0.1194,1451882.6000,595903.3982
5,dry_bean,mlp,0.4762,0.0851,0.3994,0.0829,148391.2667,131486.3227
6,dry_bean,rf,0.8893,0.0774,0.8894,0.0770,68896552.6000,26015151.5833
7,dry_bean,svm,0.6490,0.0821,0.6375,0.0826,1121423.8000,51430.1818
8,glass,knn,0.7011,0.0614,0.6839,0.0646,16552.4000,6786.0425
9,glass,mlp,0.6405,0.0609,0.6111,0.0550,165909.9333,265660.8744


In [9]:
# Merge results
results_df = pd.concat([dict_wisard_results, sklearn_results])
results_df = results_df.sort_values(by=["dataset", "model"]).reset_index(drop=True)
results_df.sample(n=3)

,dataset,model,accuracy,accuracy_std,f1,f1_std,model_size,model_size_std
60,vehicle,Dict-Wisard,0.8672,0.0368,0.8649,0.0383,353672.5333,221462.6439
56,sepsis,knn,0.9256,0.0009,0.8910,0.0003,4809122.2000,53816.6679
2,breast_cancer,mlp,0.9315,0.0224,0.9308,0.0241,123549.4667,180986.9919


In [10]:
# Add metric column based on dataset info

dfs = []

for _, row in datasets_info.iterrows():
    df = results_df[results_df["dataset"] == row["dataset_name"]].copy()
    if row["metric"] == "f1 weighted":
        metric = "f1"
        metric_std = "f1_std"
    else:
        metric = "accuracy"
        metric_std = "accuracy_std"
    
    df["metric"] = df[metric]
    df["metric_std"] = df[metric_std]
    df["performance_metric"] = metric
    dfs.append(df.reset_index(drop=True))

results_df = pd.concat(dfs).reset_index(drop=True)

# Some beautify
results_df.dataset = results_df.dataset.str.replace("_", " ")
results_df.dataset = results_df.dataset.str.title()
results_df

results_df.to_csv("results.csv", index=False)
print("Results saved to results.csv")

Results saved to results.csv


### Relative performance (per dataset, normalized by model with best metric value)

In [11]:
relative_results_df = results_df.copy()


for dset, df in relative_results_df.groupby("dataset"):
    highest_metric = df["metric"].idxmax()
    
    for metric in ["accuracy", "f1", "model_size", "metric"]:
        relative_results_df.loc[df.index, f"{metric}_relative"] = df[metric] / df.loc[highest_metric, metric]
        
relative_results_df.to_csv("results_relative.csv", index=False)
print("Results saved to results_relative.csv")

Results saved to results_relative.csv


In [12]:
relative_results_df.sample(n=3)

,dataset,model,accuracy,accuracy_std,f1,f1_std,model_size,model_size_std,metric,metric_std,performance_metric,accuracy_relative,f1_relative,model_size_relative,metric_relative
74,Yeast,svm,0.5856,0.0158,0.5773,0.0196,108023.2000,5861.3365,0.5773,0.0196,f1,0.9964,0.9951,1.1260,0.9951
9,Dry Bean,svm,0.6490,0.0821,0.6375,0.0826,1121423.8000,51430.1818,0.6375,0.0826,f1,0.7202,0.7074,1.6287,0.7074
35,Optical Handwritten,Dict-Wisard,0.9714,0.0083,0.9714,0.0083,8721863.2667,7681296.4388,0.9714,0.0083,f1,0.9808,0.9808,17.9269,0.9808


# 2. Wisard is competitive with classical machine learning algorithms

### Size and performance

In [13]:

def add_mean_line(df):
    line = {"dataset": "Mean"}
    for c in df.columns:
        if c != "dataset":
            line[c] = df[c].mean()
    df.loc[len(df)] = line
    return df

def add_median_line(df):
    line = {"dataset": "Median"}
    for c in df.columns:
        if c != "dataset":
            line[c] = df[c].median()
    df.loc[len(df)] = line
    return df

def raw_relative_table(df, raw_metric, relative_metric, order_of_models: List[str]=None):
    # Pivot the DataFrame to create the raw metric table and relative table
    raw_df = (
        df.pivot(index="dataset", columns="model", values=raw_metric)
        .rename_axis(None, axis=1)
        .reset_index()
    )
    if order_of_models:
        raw_df = raw_df[["dataset"] + order_of_models]
    raw_df = add_mean_line(raw_df)
    raw_df = add_median_line(raw_df)
    raw_df.set_index("dataset", inplace=True)

    relative_df = (
        df.pivot(
            index="dataset", columns="model", values=relative_metric
        )
        .rename_axis(None, axis=1)
        .reset_index()
    )
    if order_of_models:
        relative_df = relative_df[["dataset"] + order_of_models]

    relative_df = add_mean_line(relative_df)
    relative_df = add_median_line(relative_df)
    relative_df.set_index("dataset", inplace=True)
    
    # Concatenating the DataFrames
    final_df = pd.concat([raw_df, relative_df], axis=1)

    final_df.columns = pd.MultiIndex.from_product(
        [["Absolute", "Relative"], raw_df.columns.str.split("_").str[0]]
    )
    return final_df

In [14]:
df = pd.read_csv("results_relative.csv")
df.loc[df["model"] == "knn", "model"] = "KNN"
df.loc[df["model"] == "rf", "model"] = "RF"
df.loc[df["model"] == "mlp", "model"] = "MLP"
# df.loc[df["model"] == "mlp2", "model"] = "MLP"
df.loc[df["model"] == "svm", "model"] = "SVM"

# Due to NAN values
# df = df[df["model"] != "SVM"]
# order_of_models = ["Dict-Wisard",  "RF", "KNN", "MLP-1L", "MLP-2L"]

order_of_models = ["Dict-Wisard", "SVM", "RF", "KNN", "MLP"]

performance_df = raw_relative_table(df, "metric", "metric_relative", order_of_models)
# order_of_datasets = performance_df["Relative"]["Dict-Wisard"].sort_values(ascending=False).keys().to_list()
# order_of_datasets.remove("Mean")
# order_of_datasets.append("Mean")
# performance_df.index = order_of_datasets
write_latex_table("performance_table.tex", performance_df.to_latex(float_format="%.2f"))
performance_df

Table written to: tables/performance_table.tex


/tmp/ipykernel_2961821/3078159784.py:19: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  write_latex_table("performance_table.tex", performance_df.to_latex(float_format="%.2f"))


Absolute                                Relative  \
                    Dict-Wisard    SVM     RF    KNN    MLP Dict-Wisard   
dataset                                                                   
Breast Cancer            0.9274 0.9552 0.9554 0.9288 0.9308      0.9708   
Dry Bean                 0.9012 0.6375 0.8894 0.6777 0.3994      1.0000   
Glass                    0.5661 0.6460 0.7505 0.6839 0.6111      0.7544   
Image Segmentation       0.6540 0.8952 0.9270 0.7143 0.8413      0.7055   
Iris                     0.9600 0.9400 0.9556 0.9467 0.9556      1.0000   
Letter                   0.8739 0.9626 0.9594 0.9290 0.8846      0.9079   
Motion Sense             0.7314 0.8342 0.9150 0.7307 0.8439      0.7994   
Optical Handwritten      0.9714 0.9904 0.9827 0.9867 0.9850      0.9808   
Rice                     0.9051 0.9304 0.9255 0.8804 0.8433      0.9728   
Satimage                 0.8847 0.9068 0.9090 0.9038 0.9036      0.9733   
Segment                  0.8698 0.9593 0.9760 0.9541 0.9673      0.8912   
Sepsis                   0.8883 0.8909 0.8909 0.8910 0.8909      0.9969   
Vehicle                  0.8672 0.9386 0.9547 0.8428 0.9197      0.9084   
Wine                     0.9339 0.9434 0.9810 0.7088 0.9356      0.9521   
Yeast                    0.4874 0.5773 0.5695 0.5795 0.5801      0.8402   
Mean                     0.8281 0.8672 0.9028 0.8239 0.8328      0.9102   
Median                   0.8793 0.9186 0.9263 0.8616 0.8877      0.9312   

                                                 
                       SVM     RF    KNN    MLP  
dataset                                          
Breast Cancer       0.9998 1.0000 0.9722 0.9743  
Dry Bean            0.7074 0.9868 0.7520 0.4432  
Glass               0.8608 1.0000 0.9114 0.8143  
Image Segmentation  0.9658 1.0000 0.7705 0.9075  
Iris                0.9792 0.9954 0.9861 0.9954  
Letter              1.0000 0.9968 0.9651 0.9190  
Motion Sense        0.9118 1.0000 0.7986 0.9224  
Optical Handwritten 1.0000 0.9923 0.9962 0.9945  
Rice                1.0000 0.9948 0.9463 0.9064  
Satimage            0.9976 1.0000 0.9943 0.9942  
Segment             0.9829 1.0000 0.9775 0.9911  
Sepsis              0.9999 0.9998 1.0000 0.9999  
Vehicle             0.9831 1.0000 0.8828 0.9633  
Wine                0.9617 1.0000 0.7225 0.9537  
Yeast               0.9951 0.9816 0.9988 1.0000  
Mean                0.9563 0.9965 0.9116 0.9186  
Median              0.9830 0.9999 0.9557 0.9585

In [15]:
size_df = raw_relative_table(df, "model_size", "model_size_relative", order_of_models)
size_df["Absolute"] = size_df["Absolute"] / 1024
# size_df.index = order_of_datasets
write_latex_table("size_table.tex", size_df.to_latex(float_format="%.2f"))
size_df

Table written to: tables/size_table.tex


/tmp/ipykernel_2961821/2098181408.py:4: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  write_latex_table("size_table.tex", size_df.to_latex(float_format="%.2f"))


Absolute                                             \
                    Dict-Wisard       SVM          RF        KNN       MLP   
dataset                                                                      
Breast Cancer           39.6299   11.1393     46.5656   126.5945  120.6538   
Dry Bean               672.3861 1095.1404  67281.7896  1417.8541  144.9133   
Glass                   36.9090   12.0305   1059.5763    16.1645  162.0214   
Image Segmentation     105.0223   11.5131    224.7023    26.1680  263.7539   
Iris                     3.3380    1.8996    205.5102     5.3145  119.2204   
Letter                7025.0604 1727.7078 718892.0286  3150.7422   76.6546   
Motion Sense         45799.3444 6427.4793  52481.6523 11419.0967 1020.8035   
Optical Handwritten   8517.4446  475.1211  67937.0917  2584.4672  398.6618   
Rice                    23.9501   24.9316   2820.9688   252.7270  121.3453   
Satimage              6359.9932  349.5371  32825.0337  1444.1305  195.5293   
Segment                118.1079   47.8273   2394.3191   231.6334  407.6588   
Sepsis                  11.1712  450.2906  24890.3230  4696.4084   14.1017   
Vehicle                345.3833   19.7625   1923.2629    80.9426  348.6048   
Wine                    50.4378    3.5791    270.7033    18.5373  600.9339   
Yeast                  422.2283  105.4914  25183.9584    81.4973   93.6893   
Mean                  4635.3604  717.5634  66562.4991  1703.4852  272.5697   
Median                 231.7456   76.6594  13855.6459   242.1802  178.7754   

                       Relative                                  
                    Dict-Wisard    SVM       RF     KNN     MLP  
dataset                                                          
Breast Cancer            0.8511 0.2392   1.0000  2.7186  2.5911  
Dry Bean                 1.0000 1.6287 100.0642  2.1087  0.2155  
Glass                    0.0348 0.0114   1.0000  0.0153  0.1529  
Image Segmentation       0.4674 0.0512   1.0000  0.1165  1.1738  
Iris                     1.0000 0.5691  61.5677  1.5921 35.7166  
Letter                   4.0661 1.0000 416.0958  1.8237  0.0444  
Motion Sense             0.8727 0.1225   1.0000  0.2176  0.0195  
Optical Handwritten     17.9269 1.0000 142.9890  5.4396  0.8391  
Rice                     0.9606 1.0000 113.1481 10.1368  4.8671  
Satimage                 0.1938 0.0106   1.0000  0.0440  0.0060  
Segment                  0.0493 0.0200   1.0000  0.0967  0.1703  
Sepsis                   0.0024 0.0959   5.2999  1.0000  0.0030  
Vehicle                  0.1796 0.0103   1.0000  0.0421  0.1813  
Wine                     0.1863 0.0132   1.0000  0.0685  2.2199  
Yeast                    4.5067 1.1260 268.8031  0.8699  1.0000  
Mean                     2.1532 0.4599  74.3979  1.7527  3.2800  
Median                   0.8619 0.1808   3.1499  0.9349  0.5273

### Pareto frontier

In [16]:
# Identify Pareto frontier
def is_pareto_efficient(costs):
    is_efficient = np.ones(costs.shape[0], dtype=bool)
    for i, c in enumerate(costs):
        if is_efficient[i]:
            is_efficient[is_efficient] = np.any(costs[is_efficient] < c, axis=1)
            is_efficient[i] = True  # Keep the current point
    return is_efficient

for dset_name, dset_df in df.groupby("dataset"):
    costs = dset_df[["model_size", "metric"]].to_numpy()
    # Invert metric (lower is better)
    costs[:, 1] = 1 / costs[:, 1]
    pareto = is_pareto_efficient(costs)
    df.loc[dset_df.index, "pareto"] = pareto
    
df.to_csv("results_relative_pareto.csv", index=False)
print(f"Csv written to results_relative_pareto.csv")
df.head(n=12)

Csv written to results_relative_pareto.csv


,dataset,model,accuracy,accuracy_std,f1,f1_std,model_size,model_size_std,metric,metric_std,performance_metric,accuracy_relative,f1_relative,model_size_relative,metric_relative,pareto
0,Breast Cancer,Dict-Wisard,0.9279,0.0159,0.9274,0.0163,40581.0000,23921.9804,0.9274,0.0163,f1,0.9712,0.9708,0.8511,0.9708,False
1,Breast Cancer,KNN,0.9297,0.0255,0.9288,0.0269,129632.8000,52987.9426,0.9288,0.0269,f1,0.9731,0.9722,2.7186,0.9722,False
2,Breast Cancer,MLP,0.9315,0.0224,0.9308,0.0241,123549.4667,180986.9919,0.9308,0.0241,f1,0.9749,0.9743,2.5911,0.9743,False
3,Breast Cancer,RF,0.9554,0.0166,0.9554,0.0167,47683.1333,38508.2852,0.9554,0.0167,f1,1.0000,1.0000,1.0000,1.0000,True
4,Breast Cancer,SVM,0.9561,0.0397,0.9552,0.0417,11406.6000,4215.2497,0.9552,0.0417,f1,1.0007,0.9998,0.2392,0.9998,True
5,Dry Bean,Dict-Wisard,0.9011,0.0047,0.9012,0.0043,688523.4000,481035.0196,0.9012,0.0043,f1,1.0000,1.0000,1.0000,1.0000,True
6,Dry Bean,KNN,0.6769,0.1227,0.6777,0.1194,1451882.6000,595903.3982,0.6777,0.1194,f1,0.7513,0.7520,2.1087,0.7520,False
7,Dry Bean,MLP,0.4762,0.0851,0.3994,0.0829,148391.2667,131486.3227,0.3994,0.0829,f1,0.5284,0.4432,0.2155,0.4432,True
8,Dry Bean,RF,0.8893,0.0774,0.8894,0.0770,68896552.6000,26015151.5833,0.8894,0.0770,f1,0.9869,0.9868,100.0642,0.9868,False
9,Dry Bean,SVM,0.6490,0.0821,0.6375,0.0826,1121423.8000,51430.1818,0.6375,0.0826,f1,0.7202,0.7074,1.6287,0.7074,False


In [17]:
df.groupby("model").pareto.value_counts().sort_index().to_frame()

pareto
model       pareto        
Dict-Wisard False       11
            True         4
KNN         False       12
            True         3
MLP         False        7
            True         8
RF          False        7
            True         8
SVM         False        4
            True        11

In [25]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from math import cos, sin, radians

# Assuming 'data' is our DataFrame
data = df.copy()

# dfs = []
# for dset in order_of_datasets:
#     x = data[data["dataset"] == dset].copy()
#     dfs.append(x)
# data = pd.concat(dfs).reset_index(drop=True)

# Define marker symbols for each model
marker_symbols = {
    "KNN": "triangle-up-open",
    "MLP": "square-open",
    # "MLP-2L": "circle-open",
    "RF": "circle-open",
    "SVM": "diamond-open",
    "Dict-Wisard": "star-open",
}

# Define model names for legend
model_names = {
    "KNN": "KNN",
    "MLP": "MLP",
    # "MLP-2L": "MLP-2L",
    "RF": "Random Forest",
    "SVM": "SVM",
    "Dict-Wisard": "Wisard (Dict)",
}

pareto_colors = {
    True: px.colors.qualitative.Plotly[1],
    False: px.colors.qualitative.Plotly[0],
}

rows = 5
cols = 3
datasets = data["dataset"].unique().tolist()
if "Mean" in datasets:
    datasets.remove("Mean")
if "Median" in datasets:
    datasets.remove("Median")


fig = make_subplots(
    rows=rows,
    cols=cols,
    subplot_titles=datasets,
    shared_xaxes=False,
    shared_yaxes=False,
    vertical_spacing=0.05,
    horizontal_spacing=0.05,
)

positions = [
    ("top", "left"),
    ("top", "right"),
    ("top", "left"),
    ("top", "right"),
    ("top", "left"),
    ("top", "right"),
]

for i, dset in enumerate(datasets):
    row = i // cols + 1
    col = i % cols + 1

    dset_df = data[data["dataset"] == dset]

    prev_positions = []

    for j, (model, model_df) in enumerate(dset_df.groupby("model")):
        pareto = model_df["pareto"].iloc[0]
        color = pareto_colors[pareto]
        symbol = marker_symbols[model]
        size = (model_df["model_size"] / 1024 ** 2).iloc[0]
        size = f"{size:.2f} MB"
        

        if model == "Dict-Wisard" and pareto:
            symbol = "star"

        fig.add_trace(
            go.Scatter(
                x=model_df["model_size_relative"],
                y=model_df["metric"],
                mode="markers+text",
                marker=dict(symbol=symbol, size=10, color=color),
                showlegend=False,
                name=model_names[model],
                text=size,
                textfont=dict(family="Times New Roman", size=10),
                textposition="top center",
            ),
            row=row,
            col=col,
        )

        
    pareto_points = dset_df[dset_df["pareto"] == True].copy()
    pareto_points.sort_values(by="model_size_relative", inplace=True)
    # pareto_points = pareto_points.reset_index(drop=True)
    x = pareto_points["model_size_relative"].to_list()
    y = pareto_points["metric"].to_list()

    x.insert(0, min(dset_df["model_size_relative"]))
    y.insert(0, min(dset_df["metric"]))

    x.append(max(dset_df["model_size_relative"]))
    y.append(max(dset_df["metric"]))


    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode="lines",
            line=dict(
                color=px.colors.qualitative.Prism_r[0], width=0.75, dash="dot"
            ),
            showlegend=False,
        ),
        row=row,
        col=col,
    )

    # Add secondary x-axis
    # fig.update_xaxes(secondary_x=True, row=row, col=col, title_text="Model Size", tickvals=dset_df["model_size_relative"], ticktext=dset_df["model_size"])
    

# Manually map symbols to names in the legend
legend_labels = {
    symbol: model_names[model] for model, symbol in marker_symbols.items()
}

# Create a custom legend
custom_legend = []
for symbol, model_name in legend_labels.items():
    custom_legend.append(
        go.Scatter(
            x=[None],
            y=[None],
            mode="markers",
            marker=dict(
                symbol=symbol, size=12, color=px.colors.qualitative.Plotly[0]
            ),
            name=model_name,
        )
    )

custom_legend.append(
    go.Scatter(
        x=[None],
        y=[None],
        mode="lines",
        line=dict(
            color=px.colors.qualitative.Prism_r[0], width=0.5, dash="dot"
        ),
        name="Pareto Frontier",
    )
)

for trace in custom_legend:
    fig.add_trace(trace)

fig.update_layout(
    height=1200,
    width=1000,
    margin=dict(l=10, r=10, t=10, b=10),
    font=dict(family="Times New Roman", size=14),
    legend=dict(
        title="",  # Set title to empty string to remove the legend title
        orientation="h",
        yanchor="top",
        y=1.07,
        xanchor="center",
        x=0.5,
        traceorder="normal",  # Set trace order to normal to arrange legend entries horizontally
    ),
)

write_figure("model_metric_size.pdf", fig)

fig.show()


Figure written to: figures/model_metric_size.pdf
Filename   : model_metric_size.pdf
Latex label: model_metric_size


## 3. Non-Dict Wisard

In [26]:
def parse_wisard_config_name(row) -> str:
    """Given a row, parse the name of configuration.

    Parameters
    ----------
    row : pd.Series
        The row of the dataframe.

    Returns
    -------
    str
        The name of the configuration.
    """

    names = []
    if not pd.isna(row["num_hitters"]):
        names.append(f"NR: {row['num_hitters']}")
    if not pd.isna(row["width"]):
        names.append(f"W: {row['width']}")
    if not pd.isna(row["depth"]):
        names.append(f"D: {row['depth']}")
    if not pd.isna(row["capacity"]):
        names.append(f"C: {row['capacity']}")
    if not pd.isna(row["bucket_size"]):
        names.append(f"BS: {row['bucket_size']}")
    if not pd.isna(row["threshold"]):
        names.append(f"T: {row['threshold']}")
    if not pd.isna(row["est_elements"]):
        names.append(f"EST: {row['est_elements']}")
    if not pd.isna(row["false_positive_rate"]):
        names.append(f"FPR: {row['false_positive_rate']}")

    if names:
        names = ", ".join(names)
        return f"{row['ram']} ({names})"
    else:
        return row["ram"]


# --- Read results and add a column with the name of the configuration ---
wisard_results = pd.read_csv(results_wisard_path).drop_duplicates()

# --- Add useful columns ---
wisard_results["tuple_size"] = (
    wisard_results["resolution"] / wisard_results["tuple_resolution_factor"]
)
wisard_results["config_name"] = wisard_results.apply(
    parse_wisard_config_name, axis=1
)

# --- Select the columns of interest ---
wisard_results = wisard_results[
    [
        "dataset_name",
        "config_name",
        "val_accuracy_mean",
        "val_accuracy_std",
        "val_f1 weighted_mean",
        "val_f1 weighted_std",
        "val_model size_mean",
        "val_model size_std",
        "val_ties_mean",
        "val_ties_std",
        "test_accuracy_mean",
        "test_accuracy_std",
        "test_f1 weighted_mean",
        "test_f1 weighted_std",
        "test_model size_mean",
        "test_model size_std",
        "test_ties_mean",
        "test_ties_std",
        "tuple_size",
        "encoder",
        "resolution",
        "bleach",
        "rams per discriminator",
        "ram",
    ]
]

# --- Rename columns ---
wisard_results = wisard_results.rename(
    columns={
        "dataset_name": "dataset",
        "test_ties_mean": "ties",
        "test_ties_std": "ties_std",
        "test_accuracy_mean": "accuracy",
        "test_accuracy_std": "accuracy_std",
        "test_f1 weighted_mean": "f1",
        "test_f1 weighted_std": "f1_std",
        "test_model size_mean": "model_size",
        "test_model size_std": "model_size_std",
    }
)

# --- Add model column and drop duplicates ---
wisard_results["model"] = "Wisard"
wisard_results.drop_duplicates(inplace=True)

# Split dataset name from fold
wisard_results[["dataset", "fold"]] = wisard_results["dataset"].str.split(
    "_fold_", expand=True
)
wisard_results["fold"] = wisard_results["fold"].astype(int)
wisard_results.sample(n=2)

,dataset,config_name,val_accuracy_mean,val_accuracy_std,val_f1 weighted_mean,val_f1 weighted_std,val_model size_mean,val_model size_std,val_ties_mean,val_ties_std,...,ties,ties_std,tuple_size,encoder,resolution,bleach,rams per discriminator,ram,model,fold
12567,iris,"StreamThreshold (W: 28.0, D: 1.0, T: 960.0)",0.9583,0.0000,0.9582,0.0000,3072.0000,0.0000,1.0000,0.0000,...,3.0000,0.0000,5.0000,thermometer,10,7,8,StreamThreshold,Wisard,0
12769,rice,Dict,0.8929,0.0028,0.8933,0.0027,5974.3333,141.6765,86.3333,21.5149,...,133.3333,26.3860,14.0000,distributive-thermometer,28,40,14,Dict,Wisard,4


In [27]:
# Select best based on val_loss
lines = []
for (dataset, fold, ram), dataset_df in wisard_results.groupby([ "dataset", "fold", "ram"]):
    metric_name = datasets_info.loc[datasets_info["dataset_name"] == dataset, "metric"].iloc[0]
    line = dataset_df.sort_values(by=f"val_{metric_name}_mean", ascending=False).iloc[0]
    lines.append(line)

wisard_results = pd.DataFrame(lines)

# Aggregate fold results
wisard_results = aggregate_mean_std(
    wisard_results, 
    group_by=["dataset", "ram"],
    keys_to_aggregate=["accuracy", "f1", "model_size"]
)

dfs = []

# Add metric row
for _, row in datasets_info.iterrows():
    df = wisard_results[wisard_results["dataset"] == row["dataset_name"]].copy()
    if row["metric"] == "f1 weighted":
        metric = "f1"
        metric_std = "f1_std"
    else:
        metric = "accuracy"
        metric_std = "accuracy_std"
    
    df["metric"] = df[metric]
    df["metric_std"] = df[metric_std]
    df["performance_metric"] = metric
    dfs.append(df.reset_index(drop=True))

wisard_results = pd.concat(dfs).reset_index(drop=True)

# Put dataset name in Camel case
wisard_results.dataset = wisard_results.dataset.str.replace("_", " ")
wisard_results.dataset = wisard_results.dataset.str.title()
wisard_results

,dataset,ram,accuracy,f1,model_size,accuracy_std,f1_std,model_size_std,metric,metric_std,performance_metric
0,Breast Cancer,CountMinSketch,0.9367,0.9365,89990.4000,0.0237,0.0237,100764.3960,0.9365,0.0237,f1
1,Breast Cancer,CountingBloomFilter,0.9080,0.9081,563136.0000,0.0503,0.0492,512378.1803,0.9081,0.0492,f1
2,Breast Cancer,CountingCuckoo,0.9326,0.9324,283248.0000,0.0266,0.0264,427504.4219,0.9324,0.0264,f1
3,Breast Cancer,Dict,0.9279,0.9274,40581.0000,0.0159,0.0163,23921.9804,0.9274,0.0163,f1
4,Breast Cancer,HeavyHitters,0.9220,0.9219,187296.0000,0.0291,0.0293,178812.2895,0.9219,0.0293,f1
...,...,...,...,...,...,...,...,...,...,...,...
85,Yeast,CountingBloomFilter,0.5411,0.5272,459648.0000,0.0199,0.0226,351968.9877,0.5272,0.0226,f1
86,Yeast,CountingCuckoo,0.5384,0.5209,1498949.3333,0.0277,0.0294,2383408.3700,0.5209,0.0294,f1
87,Yeast,Dict,0.5091,0.4874,432361.8000,0.0605,0.0695,185991.4349,0.4874,0.0695,f1
88,Yeast,HeavyHitters,0.5458,0.5307,370432.0000,0.0193,0.0179,180401.9122,0.5307,0.0179,f1


In [28]:
relative_results_df = wisard_results.copy()


for dset, df in relative_results_df.groupby("dataset"):
    highest_metric = df.loc[df["ram"]== "Dict", "metric"].idxmax()
    
    for metric in ["accuracy", "f1", "model_size", "metric"]:
        relative_results_df.loc[df.index, f"{metric}_relative"] = df[metric] / df.loc[highest_metric, metric]
        
relative_results_df["model"] = wisard_results["ram"]
relative_results_df.to_csv("results_relative_bloom.csv", index=False)
print("Results saved to results_relative_bloom.csv")

relative_results_df

Results saved to results_relative_bloom.csv


,dataset,ram,accuracy,f1,model_size,accuracy_std,f1_std,model_size_std,metric,metric_std,performance_metric,accuracy_relative,f1_relative,model_size_relative,metric_relative,model
0,Breast Cancer,CountMinSketch,0.9367,0.9365,89990.4000,0.0237,0.0237,100764.3960,0.9365,0.0237,f1,1.0094,1.0097,2.2176,1.0097,CountMinSketch
1,Breast Cancer,CountingBloomFilter,0.9080,0.9081,563136.0000,0.0503,0.0492,512378.1803,0.9081,0.0492,f1,0.9786,0.9792,13.8768,0.9792,CountingBloomFilter
2,Breast Cancer,CountingCuckoo,0.9326,0.9324,283248.0000,0.0266,0.0264,427504.4219,0.9324,0.0264,f1,1.0050,1.0054,6.9798,1.0054,CountingCuckoo
3,Breast Cancer,Dict,0.9279,0.9274,40581.0000,0.0159,0.0163,23921.9804,0.9274,0.0163,f1,1.0000,1.0000,1.0000,1.0000,Dict
4,Breast Cancer,HeavyHitters,0.9220,0.9219,187296.0000,0.0291,0.0293,178812.2895,0.9219,0.0293,f1,0.9936,0.9941,4.6154,0.9941,HeavyHitters
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Yeast,CountingBloomFilter,0.5411,0.5272,459648.0000,0.0199,0.0226,351968.9877,0.5272,0.0226,f1,1.0628,1.0816,1.0631,1.0816,CountingBloomFilter
86,Yeast,CountingCuckoo,0.5384,0.5209,1498949.3333,0.0277,0.0294,2383408.3700,0.5209,0.0294,f1,1.0575,1.0687,3.4669,1.0687,CountingCuckoo
87,Yeast,Dict,0.5091,0.4874,432361.8000,0.0605,0.0695,185991.4349,0.4874,0.0695,f1,1.0000,1.0000,1.0000,1.0000,Dict
88,Yeast,HeavyHitters,0.5458,0.5307,370432.0000,0.0193,0.0179,180401.9122,0.5307,0.0179,f1,1.0720,1.0888,0.8568,1.0888,HeavyHitters


In [29]:
performance_df = raw_relative_table(relative_results_df, "metric", "metric_relative")
write_latex_table("performance_table_bloom.tex", performance_df.to_latex(float_format="%.2f"))
performance_df

Table written to: tables/performance_table_bloom.tex


/tmp/ipykernel_2961821/45709815.py:2: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



Absolute                                            \
                    CountMinSketch CountingBloomFilter CountingCuckoo   Dict   
dataset                                                                        
Breast Cancer               0.9365              0.9081         0.9324 0.9274   
Dry Bean                    0.8247              0.8864         0.7951 0.9012   
Glass                       0.5951              0.5823         0.5786 0.5661   
Image Segmentation          0.7635              0.8476         0.8222 0.6540   
Iris                        0.9089              0.9533         0.9311 0.9600   
Letter                      0.8782              0.8891         0.8767 0.8739   
Motion Sense                0.5654              0.6783         0.6845 0.7314   
Optical Handwritten         0.9715              0.9731         0.9741 0.9714   
Rice                        0.8968              0.9082         0.9139 0.9051   
Satimage                    0.8764              0.8765         0.8802 0.8847   
Segment                     0.8820              0.8677         0.8759 0.8698   
Sepsis                      0.8907              0.8613         0.8882 0.8883   
Vehicle                     0.8601              0.8401         0.8614 0.8672   
Wine                        0.8940              0.9090         0.9199 0.9339   
Yeast                       0.5247              0.5272         0.5209 0.4874   
Mean                        0.8179              0.8339         0.8303 0.8281   
Median                      0.8773              0.8721         0.8763 0.8793   

                                                       Relative  \
                    HeavyHitters StreamThreshold CountMinSketch   
dataset                                                           
Breast Cancer             0.9219          0.9120         1.0097   
Dry Bean                  0.7612          0.8206         0.9151   
Glass                     0.5755          0.5602         1.0512   
Image Segmentation        0.8143          0.7063         1.1675   
Iris                      0.9467          0.9378         0.9468   
Letter                    0.8608          0.8658         1.0049   
Motion Sense              0.6882          0.6040         0.7730   
Optical Handwritten       0.9649          0.9623         1.0001   
Rice                      0.9044          0.9127         0.9908   
Satimage                  0.8796          0.8653         0.9906   
Segment                   0.8913          0.8719         1.0139   
Sepsis                    0.8845          0.8796         1.0027   
Vehicle                   0.8550          0.8720         0.9918   
Wine                      0.9159          0.9195         0.9572   
Yeast                     0.5307          0.4988         1.0765   
Mean                      0.8263          0.8126         0.9928   
Median                    0.8702          0.8688         0.9965   

                                                                            \
                    CountingBloomFilter CountingCuckoo   Dict HeavyHitters   
dataset                                                                      
Breast Cancer                    0.9792         1.0054 1.0000       0.9941   
Dry Bean                         0.9835         0.8822 1.0000       0.8446   
Glass                            1.0286         1.0220 1.0000       1.0165   
Image Segmentation               1.2961         1.2573 1.0000       1.2451   
Iris                             0.9931         0.9699 1.0000       0.9861   
Letter                           1.0175         1.0032 1.0000       0.9850   
Motion Sense                     0.9275         0.9359 1.0000       0.9410   
Optical Handwritten              1.0018         1.0027 1.0000       0.9933   
Rice                             1.0034         1.0098 1.0000       0.9992   
Satimage                         0.9907         0.9949 1.0000       0.9943   
Segment                          0.9976         1.0070 1.0000       1.

In [31]:
size_df = raw_relative_table(relative_results_df, "model_size", "model_size_relative")
size_df["Absolute"] = size_df["Absolute"] / 1024 ** 2
write_latex_table("size_table_bloom.tex", size_df.to_latex(float_format="%.2f"))
size_df

Table written to: tables/size_table_bloom.tex


/tmp/ipykernel_2961821/1480569691.py:3: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



Absolute                                             \
                    CountMinSketch CountingBloomFilter CountingCuckoo    Dict   
dataset                                                                         
Breast Cancer               0.0858              0.5370         0.2701  0.0387   
Dry Bean                    0.2584              1.0741         2.0518  0.6566   
Glass                       0.2891              0.9419         0.6170  0.0360   
Image Segmentation          0.1327              1.6098         1.3751  0.1026   
Iris                        0.0506              0.1323         0.3248  0.0033   
Letter                      1.3482              5.7995         7.8178  6.8604   
Motion Sense                8.6336             24.0255        56.4093 44.7259   
Optical Handwritten         3.1406             14.7021         9.0920  8.3178   
Rice                        0.0176              0.0847         0.0640  0.0234   
Satimage                    1.0209              2.9294         4.2631  6.2109   
Segment                     0.2702              0.7449         2.7432  0.1153   
Sepsis                      0.0275              0.0424         0.2821  0.0109   
Vehicle                     0.2146              0.1891         0.6925  0.3373   
Wine                        0.0501              0.2211         0.7636  0.0493   
Yeast                       0.4001              0.4384         1.4295  0.4123   
Mean                        1.0627              3.5648         5.8797  4.5267   
Median                      0.2643              0.8434         1.4023  0.2263   

                                                       Relative  \
                    HeavyHitters StreamThreshold CountMinSketch   
dataset                                                           
Breast Cancer             0.1786          0.1271         2.2176   
Dry Bean                  0.3413          0.5197         0.3935   
Glass                     0.1305          0.2427         8.0194   
Image Segmentation        0.4243          0.2477         1.2941   
Iris                      0.0347          0.0287        15.5344   
Letter                    1.2562          2.5867         0.1965   
Motion Sense              9.7888          6.8538         0.1930   
Optical Handwritten       2.6372          3.7773         0.3776   
Rice                      0.0526          0.0147         0.7512   
Satimage                  0.8919          0.8086         0.1644   
Segment                   0.4629          0.1136         2.3428   
Sepsis                    0.0257          0.0261         2.5210   
Vehicle                   0.1064          0.1566         0.6363   
Wine                      0.0860          0.1512         1.0173   
Yeast                     0.3533          0.2847         0.9704   
Mean                      1.1180          1.0626         2.4420   
Median                    0.3473          0.2452         0.9939   

                                                                            \
                    CountingBloomFilter CountingCuckoo   Dict HeavyHitters   
dataset                                                                      
Breast Cancer                   13.8768         6.9798 1.0000       4.6154   
Dry Bean                         1.6358         3.1248 1.0000       0.5198   
Glass                           26.1327        17.1178 1.0000       3.6211   
Image Segmentation              15.6964        13.4080 1.0000       4.1366   
Iris                            40.5843        99.6465 1.0000      10.6446   
Letter                           0.8454         1.1395 1.0000       0.1831   
Motion Sense                     0.5372         1.2612 1.0000       0.2189   
Optical Handwritten              1.7675         1.0931 1.0000       0.3171   
Rice                             3.6196         2.7355 1.0000       2.2473   
Satimage                         0.4717         0.6864 1.0000       0.1436   
Segment                          6.4583        23.